In [40]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
cd ..

C:\Projects\python\recommender


In [20]:
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [38]:
from datasets import TorchMovielen10k, TorchTopcoder, DataBunch
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss

In [5]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
REGS_PATH = Path("./inputs/topcoder/regs.csv")

In [6]:
db = TorchTopcoder(REGS_PATH)
db

2019-09-25 20:36:44,356 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\topcoder\regs.csv
2019-09-25 20:36:44,356 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\topcoder\regs.csv
2019-09-25 20:36:44,356 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\topcoder\regs.csv
I0925 20:36:44.356832  2240 torch_topcoder.py:47] Read dataset in inputs\topcoder\regs.csv
2019-09-25 20:36:44,363 - C:\Projects\python\recommender\utils.py - INFO - Original regs shape: (610025, 3)
2019-09-25 20:36:44,363 - C:\Projects\python\recommender\utils.py - INFO - Original regs shape: (610025, 3)
2019-09-25 20:36:44,363 - C:\Projects\python\recommender\utils.py - INFO - Original regs shape: (610025, 3)
I0925 20:36:44.363837  2240 torch_topcoder.py:48] Original regs shape: (610025, 3)
2019-09-25 20:36:44,489 - C:\Projects\python\recommender\utils.py - INFO - Original registants size: 60017
2019-09-25 20:36:44,489 - C:\Projects\python\re

In [8]:
db.config_db(batch_size=BATCH,
                      shuffle=SHUFFLE,
                      num_workers=WORKERS,
                      device=DEVICE)

In [10]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

### Create criterion

In [11]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [12]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x000001F26D1EAAE8>, 1, 1)

In [13]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x000001F26D246BF8>, 1, 1, 1)

### Train model

#### Hyper-parameter

In [14]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

#### Train FM Model

In [15]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [16]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [17]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt, step_size=DECAY_FREQ, gamma=DECAY_GAMMA)

In [19]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [21]:
fm_learner.fit(epoch=8, loss_callback=simple_loss_callback, log_dir=get_log_dir('topcoder', 'fm'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 37675.703864504314
Epoch 0 step 1: training accuarcy: 0.325
Epoch 0 step 1: training loss: 36599.736475448546
Epoch 0 step 2: training accuarcy: 0.34750000000000003
Epoch 0 step 2: training loss: 35615.31982925032
Epoch 0 step 3: training accuarcy: 0.35100000000000003
Epoch 0 step 3: training loss: 34615.4817054277
Epoch 0 step 4: training accuarcy: 0.379
Epoch 0 step 4: training loss: 33644.801296626974
Epoch 0 step 5: training accuarcy: 0.395
Epoch 0 step 5: training loss: 32720.449225403172
Epoch 0 step 6: training accuarcy: 0.43
Epoch 0 step 6: training loss: 31828.289722670477
Epoch 0 step 7: training accuarcy: 0.41200000000000003
Epoch 0 step 7: training loss: 30897.08321612749
Epoch 0 step 8: training accuarcy: 0.47200000000000003
Epoch 0 step 8: training loss: 30067.42791356266
Epoch 0 step 9: training accuarcy: 0.47800000000000004
Epoch 0 step 9: training loss: 29184.55817753484
Epoch 0 step 10: training accuarcy: 0.537
Epoch 0 step 10: 

Epoch 0 step 85: training loss: 2653.4429234977392
Epoch 0 step 86: training accuarcy: 0.9635
Epoch 0 step 86: training loss: 2550.409468341607
Epoch 0 step 87: training accuarcy: 0.9715
Epoch 0 step 87: training loss: 2469.2917007705064
Epoch 0 step 88: training accuarcy: 0.9665
Epoch 0 step 88: training loss: 2414.710345401264
Epoch 0 step 89: training accuarcy: 0.9645
Epoch 0 step 89: training loss: 2333.4486915904654
Epoch 0 step 90: training accuarcy: 0.9635
Epoch 0 step 90: training loss: 2267.7874215121824
Epoch 0 step 91: training accuarcy: 0.961
Epoch 0 step 91: training loss: 2183.3832112490873
Epoch 0 step 92: training accuarcy: 0.9685
Epoch 0 step 92: training loss: 2116.704745038552
Epoch 0 step 93: training accuarcy: 0.9715
Epoch 0 step 93: training loss: 2057.723475437113
Epoch 0 step 94: training accuarcy: 0.9715
Epoch 0 step 94: training loss: 1984.0326964775147
Epoch 0 step 95: training accuarcy: 0.9725
Epoch 0 step 95: training loss: 1936.757999925666
Epoch 0 step 96

Epoch 0 step 172: training accuarcy: 0.9945
Epoch 0 step 172: training loss: 425.0257884882394
Epoch 0 step 173: training accuarcy: 0.9945
Epoch 0 step 173: training loss: 421.4626819664196
Epoch 0 step 174: training accuarcy: 0.993
Epoch 0 step 174: training loss: 414.43250600558804
Epoch 0 step 175: training accuarcy: 0.9935
Epoch 0 step 175: training loss: 409.8280806638817
Epoch 0 step 176: training accuarcy: 0.9945
Epoch 0 step 176: training loss: 407.12696780746546
Epoch 0 step 177: training accuarcy: 0.9915
Epoch 0 step 177: training loss: 399.084360623394
Epoch 0 step 178: training accuarcy: 0.997
Epoch 0 step 178: training loss: 394.00021115296954
Epoch 0 step 179: training accuarcy: 0.9945
Epoch 0 step 179: training loss: 403.04572063116314
Epoch 0 step 180: training accuarcy: 0.9965
Epoch 0 step 180: training loss: 398.4237869793317
Epoch 0 step 181: training accuarcy: 0.993
Epoch 0 step 181: training loss: 396.7086957094257
Epoch 0 step 182: training accuarcy: 0.993
Epoch 0

Epoch 0 step 258: training accuarcy: 0.9965
Epoch 0 step 258: training loss: 322.6649725226142
Epoch 0 step 259: training accuarcy: 0.997
Epoch 0 step 259: training loss: 317.7464473078878
Epoch 0 step 260: training accuarcy: 0.999
Epoch 0 step 260: training loss: 321.80251178393627
Epoch 0 step 261: training accuarcy: 0.997
Epoch 0 step 261: training loss: 326.00470199059305
Epoch 0 step 262: training accuarcy: 0.995
Epoch 0 step 262: training loss: 244.2342229596161
Epoch 0 step 263: training accuarcy: 0.9948717948717949
Epoch 0: train loss 4955.651903589199, train accuarcy 0.8019078373908997
Epoch 0: valid loss 1073.815513813322, valid accuarcy 0.9551243185997009


...

Epoch: 7
Epoch 7 step 1841: training loss: 329.3220929687893
Epoch 7 step 1842: training accuarcy: 0.993
Epoch 7 step 1842: training loss: 316.8665933907949
Epoch 7 step 1843: training accuarcy: 0.9955
Epoch 7 step 1843: training loss: 309.30408222590887
Epoch 7 step 1844: training accuarcy: 0.9965
Epoch 7 step 1844: training loss: 316.34608045385573
Epoch 7 step 1845: training accuarcy: 0.998
Epoch 7 step 1845: training loss: 319.88587668972855
Epoch 7 step 1846: training accuarcy: 0.998
Epoch 7 step 1846: training loss: 321.54724121306856
Epoch 7 step 1847: training accuarcy: 0.9985
Epoch 7 step 1847: training loss: 312.80154667536164
Epoch 7 step 1848: training accuarcy: 0.9975
Epoch 7 step 1848: training loss: 319.02496557633333
Epoch 7 step 1849: training accuarcy: 0.9965
Epoch 7 step 1849: training loss: 311.1567180435996
Epoch 7 step 1850: training accuarcy: 0.9975
Epoch 7 step 1850: training loss: 313.3573495488681
Epoch 7 step 1851: training accuarcy: 0.997
Epoch 7 step 1851: 

Epoch 7 step 1925: training loss: 332.9843082352826
Epoch 7 step 1926: training accuarcy: 0.994
Epoch 7 step 1926: training loss: 331.4446077903759
Epoch 7 step 1927: training accuarcy: 0.9945
Epoch 7 step 1927: training loss: 325.7790233012583
Epoch 7 step 1928: training accuarcy: 0.9955
Epoch 7 step 1928: training loss: 344.44421882884114
Epoch 7 step 1929: training accuarcy: 0.9915
Epoch 7 step 1929: training loss: 333.33931172099693
Epoch 7 step 1930: training accuarcy: 0.995
Epoch 7 step 1930: training loss: 320.5830591366987
Epoch 7 step 1931: training accuarcy: 0.998
Epoch 7 step 1931: training loss: 335.8779391274442
Epoch 7 step 1932: training accuarcy: 0.9945
Epoch 7 step 1932: training loss: 316.46374005818694
Epoch 7 step 1933: training accuarcy: 0.9965
Epoch 7 step 1933: training loss: 334.56897552039834
Epoch 7 step 1934: training accuarcy: 0.9965
Epoch 7 step 1934: training loss: 323.9347197086603
Epoch 7 step 1935: training accuarcy: 0.994
Epoch 7 step 1935: training lo

Epoch 7 step 2010: training accuarcy: 0.9955
Epoch 7 step 2010: training loss: 328.935751366856
Epoch 7 step 2011: training accuarcy: 0.996
Epoch 7 step 2011: training loss: 325.5385763191102
Epoch 7 step 2012: training accuarcy: 0.9975
Epoch 7 step 2012: training loss: 337.1156319602303
Epoch 7 step 2013: training accuarcy: 0.995
Epoch 7 step 2013: training loss: 326.8206010556179
Epoch 7 step 2014: training accuarcy: 0.9955
Epoch 7 step 2014: training loss: 318.550511375767
Epoch 7 step 2015: training accuarcy: 0.996
Epoch 7 step 2015: training loss: 324.5610541237395
Epoch 7 step 2016: training accuarcy: 0.998
Epoch 7 step 2016: training loss: 336.9042692020139
Epoch 7 step 2017: training accuarcy: 0.995
Epoch 7 step 2017: training loss: 331.64046277004104
Epoch 7 step 2018: training accuarcy: 0.995
Epoch 7 step 2018: training loss: 327.00539135199955
Epoch 7 step 2019: training accuarcy: 0.9985
Epoch 7 step 2019: training loss: 330.69742573703854
Epoch 7 step 2020: training accuarc

Epoch 7 step 2094: training loss: 323.4826474826256
Epoch 7 step 2095: training accuarcy: 0.997
Epoch 7 step 2095: training loss: 333.4686184369758
Epoch 7 step 2096: training accuarcy: 0.9965
Epoch 7 step 2096: training loss: 339.2355380450729
Epoch 7 step 2097: training accuarcy: 0.9975
Epoch 7 step 2097: training loss: 341.9979638092247
Epoch 7 step 2098: training accuarcy: 0.995
Epoch 7 step 2098: training loss: 330.81652240053904
Epoch 7 step 2099: training accuarcy: 0.995
Epoch 7 step 2099: training loss: 319.15309088569927
Epoch 7 step 2100: training accuarcy: 0.9945
Epoch 7 step 2100: training loss: 312.6963901203337
Epoch 7 step 2101: training accuarcy: 0.9965
Epoch 7 step 2101: training loss: 321.20695039244686
Epoch 7 step 2102: training accuarcy: 0.9965
Epoch 7 step 2102: training loss: 317.5384766887032
Epoch 7 step 2103: training accuarcy: 0.9975
Epoch 7 step 2103: training loss: 242.85923543437607
Epoch 7 step 2104: training accuarcy: 0.9910256410256411
Epoch 7: train lo

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [15:26<00:00, 113.57s/it]


In [22]:
del fm_model
T.cuda.empty_cache()

### Train HRM FM Model

In [23]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [24]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [25]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [26]:
hrm_learner.fit(epoch=8,
                loss_callback=simple_loss_callback,
                log_dir=get_log_dir('topcoder', 'hrm'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 37508.48394862066
Epoch 0 step 1: training accuarcy: 0.489
Epoch 0 step 1: training loss: 36442.54428888903
Epoch 0 step 2: training accuarcy: 0.49
Epoch 0 step 2: training loss: 35401.56785084879
Epoch 0 step 3: training accuarcy: 0.4995
Epoch 0 step 3: training loss: 34378.68944804763
Epoch 0 step 4: training accuarcy: 0.5035000000000001
Epoch 0 step 4: training loss: 33382.10234287656
Epoch 0 step 5: training accuarcy: 0.5075000000000001
Epoch 0 step 5: training loss: 32409.860371521718
Epoch 0 step 6: training accuarcy: 0.504
Epoch 0 step 6: training loss: 31457.042214567897
Epoch 0 step 7: training accuarcy: 0.496
Epoch 0 step 7: training loss: 30524.22628953661
Epoch 0 step 8: training accuarcy: 0.496
Epoch 0 step 8: training loss: 29613.44862392991
Epoch 0 step 9: training accuarcy: 0.5225
Epoch 0 step 9: training loss: 28726.668180471803
Epoch 0 step 10: training accuarcy: 0.5075000000000001
Epoch 0 step 10: training loss: 27861.619561225

Epoch 0 step 87: training loss: 2623.9082366251396
Epoch 0 step 88: training accuarcy: 0.5555
Epoch 0 step 88: training loss: 2566.6791349567147
Epoch 0 step 89: training accuarcy: 0.5655
Epoch 0 step 89: training loss: 2512.556097300625
Epoch 0 step 90: training accuarcy: 0.5640000000000001
Epoch 0 step 90: training loss: 2461.505345053759
Epoch 0 step 91: training accuarcy: 0.578
Epoch 0 step 91: training loss: 2411.553838109179
Epoch 0 step 92: training accuarcy: 0.551
Epoch 0 step 92: training loss: 2364.202697669666
Epoch 0 step 93: training accuarcy: 0.5565
Epoch 0 step 93: training loss: 2317.773404728965
Epoch 0 step 94: training accuarcy: 0.5690000000000001
Epoch 0 step 94: training loss: 2276.52178179392
Epoch 0 step 95: training accuarcy: 0.5235
Epoch 0 step 95: training loss: 2232.1307986384318
Epoch 0 step 96: training accuarcy: 0.5630000000000001
Epoch 0 step 96: training loss: 2192.625100020993
Epoch 0 step 97: training accuarcy: 0.5640000000000001
Epoch 0 step 97: train

Epoch 0 step 172: training accuarcy: 0.6435
Epoch 0 step 172: training loss: 1395.9451786591947
Epoch 0 step 173: training accuarcy: 0.6495
Epoch 0 step 173: training loss: 1395.700017244339
Epoch 0 step 174: training accuarcy: 0.633
Epoch 0 step 174: training loss: 1394.4504209212423
Epoch 0 step 175: training accuarcy: 0.6525
Epoch 0 step 175: training loss: 1394.3784052799422
Epoch 0 step 176: training accuarcy: 0.633
Epoch 0 step 176: training loss: 1393.6568341847271
Epoch 0 step 177: training accuarcy: 0.6415
Epoch 0 step 177: training loss: 1392.4720800108526
Epoch 0 step 178: training accuarcy: 0.669
Epoch 0 step 178: training loss: 1391.5860948089157
Epoch 0 step 179: training accuarcy: 0.6575
Epoch 0 step 179: training loss: 1391.6421523297481
Epoch 0 step 180: training accuarcy: 0.6375000000000001
Epoch 0 step 180: training loss: 1391.7604724936352
Epoch 0 step 181: training accuarcy: 0.644
Epoch 0 step 181: training loss: 1390.3580626694768
Epoch 0 step 182: training accuar

Epoch 0 step 257: training loss: 1383.2192283649413
Epoch 0 step 258: training accuarcy: 0.6910000000000001
Epoch 0 step 258: training loss: 1383.1231538035497
Epoch 0 step 259: training accuarcy: 0.6945
Epoch 0 step 259: training loss: 1382.7370542033436
Epoch 0 step 260: training accuarcy: 0.6945
Epoch 0 step 260: training loss: 1383.1388806180219
Epoch 0 step 261: training accuarcy: 0.6950000000000001
Epoch 0 step 261: training loss: 1383.837933860504
Epoch 0 step 262: training accuarcy: 0.682
Epoch 0 step 262: training loss: 542.5687598480471
Epoch 0 step 263: training accuarcy: 0.7
Epoch 0: train loss 5339.627408337506, train accuarcy 0.6063984632492065
Epoch 0: valid loss 1365.690356413305, valid accuarcy 0.7049726843833923


...

Epoch: 7
Epoch 7 step 1841: training loss: 1382.7192701712631
Epoch 7 step 1842: training accuarcy: 0.7020000000000001
Epoch 7 step 1842: training loss: 1382.7357072268237
Epoch 7 step 1843: training accuarcy: 0.7125
Epoch 7 step 1843: training loss: 1382.7605816459884
Epoch 7 step 1844: training accuarcy: 0.71
Epoch 7 step 1844: training loss: 1381.7718497909045
Epoch 7 step 1845: training accuarcy: 0.7125
Epoch 7 step 1845: training loss: 1382.2716846890207
Epoch 7 step 1846: training accuarcy: 0.7075
Epoch 7 step 1846: training loss: 1382.7309138310045
Epoch 7 step 1847: training accuarcy: 0.7145
Epoch 7 step 1847: training loss: 1382.247658817208
Epoch 7 step 1848: training accuarcy: 0.706
Epoch 7 step 1848: training loss: 1381.892354247379
Epoch 7 step 1849: training accuarcy: 0.7105
Epoch 7 step 1849: training loss: 1381.353685694988
Epoch 7 step 1850: training accuarcy: 0.7345
Epoch 7 step 1850: training loss: 1382.7014828403112
Epoch 7 step 1851: training accuarcy: 0.713
Epoch 

Epoch 7 step 1924: training accuarcy: 0.683
Epoch 7 step 1924: training loss: 1382.9508611854221
Epoch 7 step 1925: training accuarcy: 0.6910000000000001
Epoch 7 step 1925: training loss: 1383.3192341240579
Epoch 7 step 1926: training accuarcy: 0.6950000000000001
Epoch 7 step 1926: training loss: 1383.2744461996829
Epoch 7 step 1927: training accuarcy: 0.6980000000000001
Epoch 7 step 1927: training loss: 1381.864013229542
Epoch 7 step 1928: training accuarcy: 0.7145
Epoch 7 step 1928: training loss: 1382.5911852878191
Epoch 7 step 1929: training accuarcy: 0.7005
Epoch 7 step 1929: training loss: 1383.1237428876707
Epoch 7 step 1930: training accuarcy: 0.704
Epoch 7 step 1930: training loss: 1382.2987460881814
Epoch 7 step 1931: training accuarcy: 0.6875
Epoch 7 step 1931: training loss: 1383.2594127302755
Epoch 7 step 1932: training accuarcy: 0.7025
Epoch 7 step 1932: training loss: 1382.8302645735441
Epoch 7 step 1933: training accuarcy: 0.7115
Epoch 7 step 1933: training loss: 1383.1

Epoch 7 step 2005: training loss: 1383.5513405245874
Epoch 7 step 2006: training accuarcy: 0.6970000000000001
Epoch 7 step 2006: training loss: 1382.574918088769
Epoch 7 step 2007: training accuarcy: 0.7015
Epoch 7 step 2007: training loss: 1383.1070883076884
Epoch 7 step 2008: training accuarcy: 0.6845
Epoch 7 step 2008: training loss: 1382.1518069010247
Epoch 7 step 2009: training accuarcy: 0.7010000000000001
Epoch 7 step 2009: training loss: 1383.5022730360529
Epoch 7 step 2010: training accuarcy: 0.6940000000000001
Epoch 7 step 2010: training loss: 1382.775110817881
Epoch 7 step 2011: training accuarcy: 0.683
Epoch 7 step 2011: training loss: 1382.6307904970793
Epoch 7 step 2012: training accuarcy: 0.6970000000000001
Epoch 7 step 2012: training loss: 1383.2086614715845
Epoch 7 step 2013: training accuarcy: 0.6895
Epoch 7 step 2013: training loss: 1383.4964817616806
Epoch 7 step 2014: training accuarcy: 0.6935
Epoch 7 step 2014: training loss: 1383.2985106309293
Epoch 7 step 2015: t

Epoch 7 step 2086: training loss: 1382.7805021995116
Epoch 7 step 2087: training accuarcy: 0.6980000000000001
Epoch 7 step 2087: training loss: 1382.8178241724138
Epoch 7 step 2088: training accuarcy: 0.6880000000000001
Epoch 7 step 2088: training loss: 1382.1992907678505
Epoch 7 step 2089: training accuarcy: 0.708
Epoch 7 step 2089: training loss: 1382.9550620688879
Epoch 7 step 2090: training accuarcy: 0.6915
Epoch 7 step 2090: training loss: 1382.8500328882108
Epoch 7 step 2091: training accuarcy: 0.6965
Epoch 7 step 2091: training loss: 1383.0008869831283
Epoch 7 step 2092: training accuarcy: 0.7075
Epoch 7 step 2092: training loss: 1382.381436803192
Epoch 7 step 2093: training accuarcy: 0.708
Epoch 7 step 2093: training loss: 1383.3437918158495
Epoch 7 step 2094: training accuarcy: 0.6895
Epoch 7 step 2094: training loss: 1382.887745604684
Epoch 7 step 2095: training accuarcy: 0.7085
Epoch 7 step 2095: training loss: 1383.0170676577698
Epoch 7 step 2096: training accuarcy: 0.6925


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [15:31<00:00, 117.59s/it]


In [27]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [28]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [29]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [30]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [31]:
prme_learner.fit(epoch=8,
                 loss_callback=simple_loss_callback,
                 log_dir=get_log_dir('topcoder', 'prme'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 37514.48137421203
Epoch 0 step 1: training accuarcy: 0.527
Epoch 0 step 1: training loss: 36473.747391143785
Epoch 0 step 2: training accuarcy: 0.5055000000000001
Epoch 0 step 2: training loss: 35454.346101311516
Epoch 0 step 3: training accuarcy: 0.5015000000000001
Epoch 0 step 3: training loss: 34454.33436229627
Epoch 0 step 4: training accuarcy: 0.484
Epoch 0 step 4: training loss: 33472.234940401846
Epoch 0 step 5: training accuarcy: 0.5105000000000001
Epoch 0 step 5: training loss: 32522.437981548668
Epoch 0 step 6: training accuarcy: 0.5055000000000001
Epoch 0 step 6: training loss: 31582.993115826408
Epoch 0 step 7: training accuarcy: 0.5135
Epoch 0 step 7: training loss: 30670.02176044439
Epoch 0 step 8: training accuarcy: 0.5155
Epoch 0 step 8: training loss: 29788.87466102316
Epoch 0 step 9: training accuarcy: 0.507
Epoch 0 step 9: training loss: 28914.7417543135
Epoch 0 step 10: training accuarcy: 0.5165
Epoch 0 step 10: training loss:

Epoch 0 step 87: training accuarcy: 0.5925
Epoch 0 step 87: training loss: 2762.5663323613303
Epoch 0 step 88: training accuarcy: 0.599
Epoch 0 step 88: training loss: 2706.2634834567843
Epoch 0 step 89: training accuarcy: 0.5640000000000001
Epoch 0 step 89: training loss: 2644.5764398264546
Epoch 0 step 90: training accuarcy: 0.5945
Epoch 0 step 90: training loss: 2588.3158711829783
Epoch 0 step 91: training accuarcy: 0.6055
Epoch 0 step 91: training loss: 2536.562902300486
Epoch 0 step 92: training accuarcy: 0.5885
Epoch 0 step 92: training loss: 2486.265884515271
Epoch 0 step 93: training accuarcy: 0.5915
Epoch 0 step 93: training loss: 2435.0182627281147
Epoch 0 step 94: training accuarcy: 0.6
Epoch 0 step 94: training loss: 2389.118089535354
Epoch 0 step 95: training accuarcy: 0.5975
Epoch 0 step 95: training loss: 2344.12374985393
Epoch 0 step 96: training accuarcy: 0.583
Epoch 0 step 96: training loss: 2300.0552278803657
Epoch 0 step 97: training accuarcy: 0.5915
Epoch 0 step 97

Epoch 0 step 171: training loss: 1403.2463424379605
Epoch 0 step 172: training accuarcy: 0.64
Epoch 0 step 172: training loss: 1402.6984465602586
Epoch 0 step 173: training accuarcy: 0.6355000000000001
Epoch 0 step 173: training loss: 1400.553143916913
Epoch 0 step 174: training accuarcy: 0.66
Epoch 0 step 174: training loss: 1400.4041205164006
Epoch 0 step 175: training accuarcy: 0.648
Epoch 0 step 175: training loss: 1397.8913055722246
Epoch 0 step 176: training accuarcy: 0.662
Epoch 0 step 176: training loss: 1397.7636800453208
Epoch 0 step 177: training accuarcy: 0.6465
Epoch 0 step 177: training loss: 1397.2701884359565
Epoch 0 step 178: training accuarcy: 0.655
Epoch 0 step 178: training loss: 1395.899843048871
Epoch 0 step 179: training accuarcy: 0.6735
Epoch 0 step 179: training loss: 1395.2441839108803
Epoch 0 step 180: training accuarcy: 0.6715
Epoch 0 step 180: training loss: 1395.9703598188728
Epoch 0 step 181: training accuarcy: 0.646
Epoch 0 step 181: training loss: 1394.

Epoch 0 step 256: training accuarcy: 0.673
Epoch 0 step 256: training loss: 1382.9072025880077
Epoch 0 step 257: training accuarcy: 0.6890000000000001
Epoch 0 step 257: training loss: 1383.3938732460037
Epoch 0 step 258: training accuarcy: 0.6945
Epoch 0 step 258: training loss: 1383.6231843877981
Epoch 0 step 259: training accuarcy: 0.7035
Epoch 0 step 259: training loss: 1383.1637076390673
Epoch 0 step 260: training accuarcy: 0.683
Epoch 0 step 260: training loss: 1383.6408113188377
Epoch 0 step 261: training accuarcy: 0.6915
Epoch 0 step 261: training loss: 1383.8668465713042
Epoch 0 step 262: training accuarcy: 0.6755
Epoch 0 step 262: training loss: 543.4043466801741
Epoch 0 step 263: training accuarcy: 0.6948717948717948
Epoch 0: train loss 5440.148758324528, train accuarcy 0.6237958073616028
Epoch 0: valid loss 1364.5724493932605, valid accuarcy 0.7122498750686646


...

Epoch: 7
Epoch 7 step 1841: training loss: 1382.0603360998941
Epoch 7 step 1842: training accuarcy: 0.7085
Epoch 7 step 1842: training loss: 1381.9663085083093
Epoch 7 step 1843: training accuarcy: 0.7030000000000001
Epoch 7 step 1843: training loss: 1382.155464600916
Epoch 7 step 1844: training accuarcy: 0.7030000000000001
Epoch 7 step 1844: training loss: 1381.8843937109814
Epoch 7 step 1845: training accuarcy: 0.713
Epoch 7 step 1845: training loss: 1383.6670187086052
Epoch 7 step 1846: training accuarcy: 0.6920000000000001
Epoch 7 step 1846: training loss: 1381.3106253079434
Epoch 7 step 1847: training accuarcy: 0.715
Epoch 7 step 1847: training loss: 1381.496890954251
Epoch 7 step 1848: training accuarcy: 0.7135
Epoch 7 step 1848: training loss: 1382.0257215977017
Epoch 7 step 1849: training accuarcy: 0.7145
Epoch 7 step 1849: training loss: 1382.4808050050588
Epoch 7 step 1850: training accuarcy: 0.7125
Epoch 7 step 1850: training loss: 1381.6245515092833
Epoch 7 step 1851: train

Epoch 7 step 1923: training loss: 1383.0912891277162
Epoch 7 step 1924: training accuarcy: 0.706
Epoch 7 step 1924: training loss: 1382.6613293629712
Epoch 7 step 1925: training accuarcy: 0.6970000000000001
Epoch 7 step 1925: training loss: 1383.015189897159
Epoch 7 step 1926: training accuarcy: 0.6915
Epoch 7 step 1926: training loss: 1383.0345461815352
Epoch 7 step 1927: training accuarcy: 0.682
Epoch 7 step 1927: training loss: 1382.1299860127394
Epoch 7 step 1928: training accuarcy: 0.704
Epoch 7 step 1928: training loss: 1382.3489006138675
Epoch 7 step 1929: training accuarcy: 0.7065
Epoch 7 step 1929: training loss: 1381.8290284077368
Epoch 7 step 1930: training accuarcy: 0.713
Epoch 7 step 1930: training loss: 1382.6977309232643
Epoch 7 step 1931: training accuarcy: 0.7015
Epoch 7 step 1931: training loss: 1383.862271691608
Epoch 7 step 1932: training accuarcy: 0.6920000000000001
Epoch 7 step 1932: training loss: 1383.1418919672076
Epoch 7 step 1933: training accuarcy: 0.6960000

Epoch 7 step 2005: training accuarcy: 0.677
Epoch 7 step 2005: training loss: 1382.4426488788642
Epoch 7 step 2006: training accuarcy: 0.7115
Epoch 7 step 2006: training loss: 1384.0143155934102
Epoch 7 step 2007: training accuarcy: 0.6895
Epoch 7 step 2007: training loss: 1381.786588647689
Epoch 7 step 2008: training accuarcy: 0.7020000000000001
Epoch 7 step 2008: training loss: 1382.724263304761
Epoch 7 step 2009: training accuarcy: 0.6945
Epoch 7 step 2009: training loss: 1383.3853444019803
Epoch 7 step 2010: training accuarcy: 0.6990000000000001
Epoch 7 step 2010: training loss: 1382.1493924876793
Epoch 7 step 2011: training accuarcy: 0.7115
Epoch 7 step 2011: training loss: 1382.429153467883
Epoch 7 step 2012: training accuarcy: 0.711
Epoch 7 step 2012: training loss: 1381.6580251088676
Epoch 7 step 2013: training accuarcy: 0.707
Epoch 7 step 2013: training loss: 1382.0272257235463
Epoch 7 step 2014: training accuarcy: 0.715
Epoch 7 step 2014: training loss: 1383.463280344703
Epoc

Epoch 7 step 2086: training accuarcy: 0.683
Epoch 7 step 2086: training loss: 1383.475166157004
Epoch 7 step 2087: training accuarcy: 0.6910000000000001
Epoch 7 step 2087: training loss: 1382.605241274103
Epoch 7 step 2088: training accuarcy: 0.7065
Epoch 7 step 2088: training loss: 1382.853922574848
Epoch 7 step 2089: training accuarcy: 0.7135
Epoch 7 step 2089: training loss: 1382.7739855277407
Epoch 7 step 2090: training accuarcy: 0.7035
Epoch 7 step 2090: training loss: 1381.6857171824581
Epoch 7 step 2091: training accuarcy: 0.7175
Epoch 7 step 2091: training loss: 1382.2089479325002
Epoch 7 step 2092: training accuarcy: 0.6975
Epoch 7 step 2092: training loss: 1384.6542699408378
Epoch 7 step 2093: training accuarcy: 0.679
Epoch 7 step 2093: training loss: 1382.7846881925848
Epoch 7 step 2094: training accuarcy: 0.7015
Epoch 7 step 2094: training loss: 1382.872421434367
Epoch 7 step 2095: training accuarcy: 0.6940000000000001
Epoch 7 step 2095: training loss: 1383.661988762389
Epo

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [16:07<00:00, 123.26s/it]


In [32]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [33]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [34]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [42]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [36]:
trans_learner.fit(epoch=8,
                  loss_callback=trans_loss_callback,
                  log_dir=get_log_dir('topcoder', 'trans'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 37807.26401193977
Epoch 0 step 1: training accuarcy: 0.4955
Epoch 0 step 1: training loss: 36749.778786841816
Epoch 0 step 2: training accuarcy: 0.504
Epoch 0 step 2: training loss: 35706.60817088014
Epoch 0 step 3: training accuarcy: 0.5
Epoch 0 step 3: training loss: 34674.50367937326
Epoch 0 step 4: training accuarcy: 0.5275
Epoch 0 step 4: training loss: 33686.3987004469
Epoch 0 step 5: training accuarcy: 0.5125
Epoch 0 step 5: training loss: 32706.96820491349
Epoch 0 step 6: training accuarcy: 0.5165
Epoch 0 step 6: training loss: 31751.5109393562
Epoch 0 step 7: training accuarcy: 0.505
Epoch 0 step 7: training loss: 30820.65553823673
Epoch 0 step 8: training accuarcy: 0.524
Epoch 0 step 8: training loss: 29908.863742371064
Epoch 0 step 9: training accuarcy: 0.522
Epoch 0 step 9: training loss: 29012.36624609563
Epoch 0 step 10: training accuarcy: 0.529
Epoch 0 step 10: training loss: 28151.494881656392
Epoch 0 step 11: training accuarcy: 0

Epoch 0 step 86: training loss: 2608.6593532551665
Epoch 0 step 87: training accuarcy: 0.7025
Epoch 0 step 87: training loss: 2544.652764805356
Epoch 0 step 88: training accuarcy: 0.6880000000000001
Epoch 0 step 88: training loss: 2468.6590752649295
Epoch 0 step 89: training accuarcy: 0.71
Epoch 0 step 89: training loss: 2423.153420795797
Epoch 0 step 90: training accuarcy: 0.6975
Epoch 0 step 90: training loss: 2361.7523445788593
Epoch 0 step 91: training accuarcy: 0.713
Epoch 0 step 91: training loss: 2310.594598037431
Epoch 0 step 92: training accuarcy: 0.6940000000000001
Epoch 0 step 92: training loss: 2254.93353079578
Epoch 0 step 93: training accuarcy: 0.712
Epoch 0 step 93: training loss: 2206.2737715561316
Epoch 0 step 94: training accuarcy: 0.7135
Epoch 0 step 94: training loss: 2179.5454209468944
Epoch 0 step 95: training accuarcy: 0.684
Epoch 0 step 95: training loss: 2129.032255605367
Epoch 0 step 96: training accuarcy: 0.6985
Epoch 0 step 96: training loss: 2075.8736270671

Epoch 0 step 172: training accuarcy: 0.7435
Epoch 0 step 172: training loss: 1142.4687222205114
Epoch 0 step 173: training accuarcy: 0.7425
Epoch 0 step 173: training loss: 1148.4498302713594
Epoch 0 step 174: training accuarcy: 0.748
Epoch 0 step 174: training loss: 1126.3964042349307
Epoch 0 step 175: training accuarcy: 0.751
Epoch 0 step 175: training loss: 1175.33577255694
Epoch 0 step 176: training accuarcy: 0.7195
Epoch 0 step 176: training loss: 1150.6492550502567
Epoch 0 step 177: training accuarcy: 0.738
Epoch 0 step 177: training loss: 1154.2970654569183
Epoch 0 step 178: training accuarcy: 0.751
Epoch 0 step 178: training loss: 1132.282888711028
Epoch 0 step 179: training accuarcy: 0.7485
Epoch 0 step 179: training loss: 1129.9585267991922
Epoch 0 step 180: training accuarcy: 0.755
Epoch 0 step 180: training loss: 1151.8311434767393
Epoch 0 step 181: training accuarcy: 0.737
Epoch 0 step 181: training loss: 1144.1199802066885
Epoch 0 step 182: training accuarcy: 0.7335
Epoch

Epoch 0 step 255: training loss: 1084.4392297650447
Epoch 0 step 256: training accuarcy: 0.751
Epoch 0 step 256: training loss: 1064.7772086371895
Epoch 0 step 257: training accuarcy: 0.7495
Epoch 0 step 257: training loss: 1078.0242551238125
Epoch 0 step 258: training accuarcy: 0.7555000000000001
Epoch 0 step 258: training loss: 1080.5240449647833
Epoch 0 step 259: training accuarcy: 0.7445
Epoch 0 step 259: training loss: 1070.4557052705234
Epoch 0 step 260: training accuarcy: 0.76
Epoch 0 step 260: training loss: 1051.8578109743326
Epoch 0 step 261: training accuarcy: 0.7795
Epoch 0 step 261: training loss: 1052.9890355081523
Epoch 0 step 262: training accuarcy: 0.7675000000000001
Epoch 0 step 262: training loss: 419.62126561278643
Epoch 0 step 263: training accuarcy: 0.7769230769230769
Epoch 0: train loss 5223.848272861555, train accuarcy 0.6741687059402466
Epoch 0: valid loss 1091.9839639880483, valid accuarcy 0.7284212708473206


...

Epoch: 7
Epoch 7 step 1841: training loss: 207.70664988120637
Epoch 7 step 1842: training accuarcy: 0.987
Epoch 7 step 1842: training loss: 201.9350882288199
Epoch 7 step 1843: training accuarcy: 0.9905
Epoch 7 step 1843: training loss: 200.32081485993274
Epoch 7 step 1844: training accuarcy: 0.9895
Epoch 7 step 1844: training loss: 211.9872521946416
Epoch 7 step 1845: training accuarcy: 0.9875
Epoch 7 step 1845: training loss: 211.92228309816176
Epoch 7 step 1846: training accuarcy: 0.9875
Epoch 7 step 1846: training loss: 201.46405688122888
Epoch 7 step 1847: training accuarcy: 0.99
Epoch 7 step 1847: training loss: 209.99295785035767
Epoch 7 step 1848: training accuarcy: 0.99
Epoch 7 step 1848: training loss: 212.418770928042
Epoch 7 step 1849: training accuarcy: 0.9915
Epoch 7 step 1849: training loss: 198.96235930242898
Epoch 7 step 1850: training accuarcy: 0.989
Epoch 7 step 1850: training loss: 210.20298751367457
Epoch 7 step 1851: training accuarcy: 0.9865
Epoch 7 step 1851: tr

Epoch 7 step 1925: training loss: 194.71575382818938
Epoch 7 step 1926: training accuarcy: 0.989
Epoch 7 step 1926: training loss: 202.36474075876674
Epoch 7 step 1927: training accuarcy: 0.9845
Epoch 7 step 1927: training loss: 195.91515723538737
Epoch 7 step 1928: training accuarcy: 0.9875
Epoch 7 step 1928: training loss: 191.8330441657476
Epoch 7 step 1929: training accuarcy: 0.9895
Epoch 7 step 1929: training loss: 201.41853329777916
Epoch 7 step 1930: training accuarcy: 0.987
Epoch 7 step 1930: training loss: 209.73906444579933
Epoch 7 step 1931: training accuarcy: 0.986
Epoch 7 step 1931: training loss: 200.4897903799368
Epoch 7 step 1932: training accuarcy: 0.99
Epoch 7 step 1932: training loss: 201.1701422003739
Epoch 7 step 1933: training accuarcy: 0.99
Epoch 7 step 1933: training loss: 191.6006720319378
Epoch 7 step 1934: training accuarcy: 0.99
Epoch 7 step 1934: training loss: 204.16885072806537
Epoch 7 step 1935: training accuarcy: 0.988
Epoch 7 step 1935: training loss: 

Epoch 7 step 2009: training loss: 189.85619358721257
Epoch 7 step 2010: training accuarcy: 0.989
Epoch 7 step 2010: training loss: 190.03284790052487
Epoch 7 step 2011: training accuarcy: 0.9855
Epoch 7 step 2011: training loss: 188.68278290596368
Epoch 7 step 2012: training accuarcy: 0.985
Epoch 7 step 2012: training loss: 192.35876979405282
Epoch 7 step 2013: training accuarcy: 0.99
Epoch 7 step 2013: training loss: 194.51239370097494
Epoch 7 step 2014: training accuarcy: 0.989
Epoch 7 step 2014: training loss: 198.96255752038684
Epoch 7 step 2015: training accuarcy: 0.991
Epoch 7 step 2015: training loss: 195.3538354390031
Epoch 7 step 2016: training accuarcy: 0.9875
Epoch 7 step 2016: training loss: 205.26478259370407
Epoch 7 step 2017: training accuarcy: 0.9865
Epoch 7 step 2017: training loss: 198.26959940048633
Epoch 7 step 2018: training accuarcy: 0.986
Epoch 7 step 2018: training loss: 199.2528931640351
Epoch 7 step 2019: training accuarcy: 0.9865
Epoch 7 step 2019: training l

Epoch 7 step 2093: training loss: 179.8702585323107
Epoch 7 step 2094: training accuarcy: 0.9905
Epoch 7 step 2094: training loss: 174.41150387827287
Epoch 7 step 2095: training accuarcy: 0.9925
Epoch 7 step 2095: training loss: 198.10309414644811
Epoch 7 step 2096: training accuarcy: 0.985
Epoch 7 step 2096: training loss: 180.61786192221035
Epoch 7 step 2097: training accuarcy: 0.99
Epoch 7 step 2097: training loss: 189.88928679214618
Epoch 7 step 2098: training accuarcy: 0.99
Epoch 7 step 2098: training loss: 189.52055193954126
Epoch 7 step 2099: training accuarcy: 0.985
Epoch 7 step 2099: training loss: 185.10759346590456
Epoch 7 step 2100: training accuarcy: 0.9885
Epoch 7 step 2100: training loss: 186.21774012971227
Epoch 7 step 2101: training accuarcy: 0.987
Epoch 7 step 2101: training loss: 188.7504374050971
Epoch 7 step 2102: training accuarcy: 0.987
Epoch 7 step 2102: training loss: 177.66305252278585
Epoch 7 step 2103: training accuarcy: 0.9925
Epoch 7 step 2103: training lo

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [17:43<00:00, 132.13s/it]


In [44]:
del trans_model
T.cuda.empty_cache()